In [126]:
import pandas as pd
import numpy as np
import re

### Import dataset

In [127]:
tweets = pd.read_json("BLM_2.json")

In [128]:
tweets = tweets.reset_index(drop=True)
tweets['long'] = 0.
tweets['lat'] = 0.
for idx, val in enumerate(tweets['geo']):
        tweets['lat'][idx] = val['coordinates'][0]
        tweets['long'][idx] = val['coordinates'][1]
tweets = tweets.drop(columns=['coordinates', 'geo'])

### From here we get lemmatize the text

In [129]:
def clean_hyperlinks(text):
    result = re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+'," ", text)
    return result

In [130]:
#tweets = tweets[tweets['lang'] == 'en']
tweets['text'] = tweets['text'].apply(clean_hyperlinks)
#tweets['text'].replace('', np.nan, inplace=True)
#tweets.dropna(subset=['text'], inplace=True)

In [131]:
def string_manipulation(df,column): 
    #extract hashtags
    df["hashtag"]  = df[column].str.findall(r'#.*?(?=\s|$)')
    #extract twitter account references
    df["accounts"] = df[column].str.findall(r'@.*?(?=\s|$)')
    return df
tweets = string_manipulation(tweets,"text")

In [132]:
tweets['hashtag'] = tweets['hashtag'].apply(lambda x: ','.join(map(str, x)))
from string import punctuation
def punctation(text): 
    result = text.replace("#", "")
    result = result.lower()
    return result
tweets['hashtag'] = tweets['hashtag'].apply(punctation)

In [133]:
def inlist(text): 
    result = text.split(",")
    return result
tweets['hashtag'] = tweets['hashtag'].apply(inlist)

In [134]:
list_of_tweets = []
for tweet in tweets['text']:
    temp = tweet
    list_of_tweets.append(temp)

In [135]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# This is the clean corpus.
doc_clean = [clean(doc).split() for doc in list_of_tweets] 
tweets['token'] = doc_clean

In [139]:
tweets.head(5)

,created_at,id,text,lang,long,lat,hashtag,accounts,token
0,2020-06-05,1268749908611403776,Motivation speakers how y’all doing\nSee it an...,en,3.39583,6.453060,"[alllivesmatter, blacklivesmatter, georgefloyd...",[@],"[motivation, speaker, y’all, see, believe, all..."
1,2020-06-05,1268750111770923010,@JustinSigel Thank You! Repost: People in our ...,en,-88.10330,30.679500,"[blacklivesmatter, georgefloyd]","[@JustinSigel, @]","[justinsigel, thank, you, repost, people, comm..."
2,2020-06-05,1268750215156240384,WE ALL RIDIN TOGETHER 👊🏾🙏🏾👊🏾 @ Blacklivesmatter,en,-74.01261,40.702841,[],[@],"[ridin, together, 👊🏾🙏🏾👊🏾, blacklivesmatter]"
3,2020-06-05,1268750277219360769,“stop killing black people” #cincinnatiprotest...,en,-84.50000,39.133333,"[cincinnatiprotest, cincinnati, blacklivesmatt...",[@],"[“stop, killing, black, people”, cincinnatipro..."
4,2020-06-05,1268750533294243840,Starting from the bottom now we here\nEnjoying...,en,3.39583,6.453060,"[billieeilish, billi, billionaire, millionaire...",[@],"[starting, bottom, enjoying, moment, billieeil..."


In [137]:
for idx, val in enumerate(tweets['created_at']):
    tweets['created_at'][idx] = str(tweets['created_at'][idx]).split()[0]

In [143]:
tweets.to_csv('tweets_BLM.csv')